# Part 3: Machine learning models

In parts 1 and 2, we demonstrated how to download a dataset and add machine learnable features. In part 3, we show how to train a machine learning model on a dataset and analyze the results.

### Scikit-Learn
This unit makes extensive use of the [scikit-learn](https://scikit-learn.org/stable/) package, an open-source python package for  machine learning. Matminer has been designed to make machine learning with scikit-learn as easy as possible. Other machine learning packages exist, such as [TensorFlow](https://www.tensorflow.org), which implement neural network architectures. These packages can also be used with `matminer` but are outside the scope of this workshop.

## Load and prepare a pre-featurized model

First, let's load a dataset that we can use for machine learning. In advance, we've added some composition and structure features to the `elastic_tensor_2015` dataset used in exercises 1 and 2.

In [ ]:
import os
from matminer.utils.io import load_dataframe_from_json

df = load_dataframe_from_json(os.path.join("resources", "elastic_tensor_2015_featurized.json"))
df.head()

We first need to split the dataset into the "target" property, and the "features" used for learning. In this model, we will be using the bulk modulus (`K_VRH`) as the target property. We use the `values` attribute of the dataframe to give the target properties a numpy array, rather than pandas `Series` object.

The machine learning algorithm can only use numerical features for training. Accordingly, we need to remove any non-numerical columns from our dataset. Additionally, we want to remove the `K_VRH` column from the set of features, as the model should not know about the target property in advance.

The dataset loaded above, includes `structure`, `formula`, and `composition` columns that were previously used to generate the machine learnable features. Let's remove them using the pandas `drop()` function, discussed in unit 1. Remember, `axis=1` indicates we are dropping columns rather than rows.

We can see all the descriptors in model using the `column` attribute.

## Try a random forest model using scikit-learn

The `scikit-learn` library makes it easy to use our generated features for training machine learning models. It implements a variety of different regression models and contains tools for cross-validation.

In the interests of time, in this example we will only trial a single model but it is good practice to trial multiple models to see which performs best for your machine learning problem. A good "starting" model is the random forest model. Let's create a random forest model.

In [ ]:
from sklearn.ensemble import RandomForestRegressor


Notice we created the model with the number of estimators (`n_estimators`) set to `100`. `n_estimators` is an example of a machine learning *hyper-parameter*. Most models contain many tunable hyper-parameters. To obtain good performance, it is necessary to fine tune these parameters for each individual machine learning problem. There is currently no simple way to know in advance what hyper-parameters will be optimal. Usually, a trial and error approach is used.

We can now train our model to use the input features (`X`) to predict the target property (`y`). This is achieved using the `fit()` function.

That's it, we have trained our first machine learning model!

## Evaluating model performance

Next, we need to assess how the model is performing. To do this, we first ask the model to predict the bulk modulus for every entry in our original dataframe.

Next, we can check the accuracy of our model by looking at the *root mean squared error* of our predictions. Scikit-learn provides a `mean_squared_error()` function to calculate the mean squared error. We then take the square-root of this to obtain our final performance metric.

In [ ]:
import numpy as np
from sklearn.metrics import mean_squared_error

mse = mean_squared_error(y, y_pred)
print('training RMSE = {:.3f} GPa'.format(np.sqrt(mse)))

An RMSE of 7.2 GPa looks very reasonable! However, as the model was trained and evaluated on exactly the same data, this is not a true estimate of how the model will perform for unseen materials (the primary purpose of machine learning studies).

### Cross validation

To obtain a more accurate estimate of prediction performance and validate that we are not over-fitting, we need to check the **cross-validation score** rather than the fitting score.

In cross-validation, the data is partitioned randomly into $n$ "splits" (in this case 10), each containing roughly the same number of samples. The model is trained on $n-1$ splits (the training set) and the model performance evaluated by comparing the actual and predicted values for the final split (the testing set). In total, this process is repeated $n$ times, such that each split is at some point used as the testing set. The cross-validation score is the average score across all testing sets.

There are a number of ways to partition the data into splits. In this example, we use the `KFold` method and select the number of splits to be 10. I.e., 90 % of the data will be used as the training set, with 10 % used as the testing set.

In [ ]:
from sklearn.model_selection import KFold



Note, we set `random_state=1` to ensure every attendee gets the same answer for their model.

Finally, obtaining the cross validation score can be automated using the Scikit-Learn `cross_val_score()` function. This function requires a machine learning model, the input features, and target property as arguments. Note, we pass the `kfold` object as the`cv` argument, to make `cross_val_score()` use the correct test/train splits.

For each split, the model will be trained from scratch, before the performance is evaualated. As we have to train and predict 10 times, cross validation can often take some time to perform. In our case, the model is quite small, so the process only takes about a minute. The final cross validation score is the average across all splits.

In [ ]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(rf, X, y, scoring='neg_mean_squared_error', cv=kfold)

rmse_scores = [np.sqrt(abs(s)) for s in scores]
print('Mean RMSE: {:.3f}'.format(np.mean(rmse_scores)))

Notice that our RMSE has almost tripled as now it reflects the true predictive power of the model. However, a root-mean-squared error of ~18 GPa is still not bad!

## Visualizing model performance

We can visualize the predictive performance of our model by plotting the our predictions against the actual value, for each sample in the test set for all test/train splits. First, we get the predicted values of the testing set for each split using the `cross_val_predict` method. This is similar to the `cross_val_score` method, except it returns the actual predictions, rather than the model score.

In [ ]:
from sklearn.model_selection import cross_val_predict



For plotting we use `PlotlyFig` module of matminer, which helps you quickly produce publication ready diagrams. `PlotlyFig` can produce many different types of plots. Explaining its use in detail is outside the scope of this tutorial but examples of the available plots are provided in the [FigRecipes section of the matminer_examples repository](https://github.com/hackingmaterials/matminer_examples/tree/master/matminer_examples/figrecipes-nb).

In [ ]:
from matminer.figrecipes.plot import PlotlyFig

pf = PlotlyFig(x_title='DFT (MP) bulk modulus (GPa)',
               y_title='Predicted bulk modulus (GPa)',
               mode='notebook')

pf.xy(xy_pairs=[(y, y_pred), ([0, 400], [0, 400])], 
      labels=df['formula'], 
      modes=['markers', 'lines'],
      lines=[{}, {'color': 'black', 'dash': 'dash'}], 
      showlegends=False)

Not too bad! However, there are definitely some outliers (you can hover over the points with your mouse to see what they are).

## Model interpretation

An important aspect of machine learning is being able to understand why a model is making certain predictions. Random forest models are particularly amenable to interpretation as they possess a `feature_importances` attribute, which contains the importance of each feature in deciding the final prediction. Let's look at the feature importances of our model.

To make sense of this, we need to know which feature each number corresponds to. We can use `PlotlyFig` to plot the importances of the 5 most important features.

In [ ]:
importances = rf.feature_importances_
included = X.columns.values
indices = np.argsort(importances)[::-1]

pf = PlotlyFig(y_title='Importance (%)',
               title='Feature by importances',
               mode='notebook')

pf.bar(x=included[indices][0:5], y=importances[indices][0:5])

As you can see, the average melting point of the elements and the volume per atom are the most important features in our model.

## Let's practice!

Now, let's practice. You'll try training and evaluating a machine learning model on a pre-trained dataset.